# Facebook Prophet

- Prophet works through use of an additive model whereby the non-linear trends in the series are fitted with the appropriate seasonality (whether daily, weekly, or yearly).
- It is designed to be robust to use and to handle seasonal trends and holidays well, shifts in the trend, and typically handles outliers well
- Prophet uses a generalized additive model, a type of regression model, to make predictions. It can accomodate non-linear smoothers applied ot the regressors.
- The model is decomposable into trend, seasonality, and holiday components.
- Prophet is curve-fitting instead of a model class like ARIMA that explicitly accounts for the temporal component of the model through autoregression.
- Prophet can accomodate expert information, so it's Bayesian-friendly.
-  It works best with time series that have strong seasonal effects and several seasons of historical data.

## Advantages of Prophet
- **1.Very fast** - Prophet is very fast. It is used in many applications across Facebook for producing reliable forecasts for planning and goal setting. 


- **2. Fully automatic** - Prophet is fully automatic. We will get a reasonable forecast on messy data with no manual effort. 


- **3. Tunable forecasts** - Prophet produces adjustable forecasts. It includes many possibilities for users to tweak and adjust forecasts. We can use human-interpretable parameters to improve the forecast by adding our domain knowledge.


- **4. Available in R or Python** - We can implement the Prophet procedure in R or Python. 



- **5. Handles seasonal variations well** - Prophet accommodates seasonality with multiple periods.



- **6. Robust to outliers** - It is robust to outliers. It handles outliers by removing them.



- **7. Robust to missing data** - Prophet is resilient to missing data.]
- **Disadvantages:**
- Prophet is not a guarantee of perfect forecasting accuracy, and it may not be the best tool for all applications.

In [1]:
!pip install --upgrade plotly
!pip install pystan==2.19.1.1
!pip install cmdstanpy==0.9.5
!pip install fbprophe

     --------------------------------------- 15.3/15.3 MB 68.6 kB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.9.0
    Uninstalling plotly-5.9.0:
      Successfully uninstalled plotly-5.9.0


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)


  copying pystan\stan\lib\stan_math\lib\boost_1.69.0\boost\proto\detail\or_n.hpp -> build\lib.win-amd64-cpython-39\pystan\stan\lib\stan_math\lib\boost_1.69.0\boost\proto\detail
  copying pystan\stan\lib\stan_math\lib\boost_1.69.0\boost\proto\detail\poly_function.hpp -> build\lib.win-amd64-cpython-39\pystan\stan\lib\stan_math\lib\boost_1.69.0\boost\proto\detail
  copying pystan\stan\lib\stan_math\lib\boost_1.69.0\boost\proto\detail\poly_function_funop.hpp -> build\lib.win-amd64-cpython-39\pystan\stan\lib\stan_math\lib\boost_1.69.0\boost\proto\detail
  copying pystan\stan\lib\stan_math\lib\boost_1.69.0\boost\proto\detail\poly_function_traits.hpp -> build\lib.win-am

     --------------------------------------- 16.2/16.2 MB 55.1 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pystan
Failed to build pystan
  Running setup.py install for pystan: started
  Running setup.py install for pystan: finished with status 'error'


ERROR: Could not find a version that satisfies the requirement fbprophe (from versions: none)
ERROR: No matching distribution found for fbprophe


In [3]:
# import prophet libraries
from fbprophet import Prophet
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation

In [ ]:
peyton = pd.read_csv('AirPassengers.csv')

In [ ]:
plt.plot(peyton['Month'], peyton['Passengers'])

In [ ]:
df_air=peyton.copy()
peyton_test=peyton[120:]
peyton=peyton[:120]

We fit the model by instantiating a new Prophet object. Any settings to the forecasting procedure are passed into the constructor. Then you call its fit method and pass in the historical dataframe. Fitting should take 1-5 seconds.

In [ ]:
prophet_model = Prophet()
prophet_model.fit(peyton)

In [ ]:
# Prophet's make_future_dataframe method makes a DataFrame that includes the historical data, as well as the predictions.
future = prophet_model.make_future_dataframe(periods=24, freq='m')
future.tail()

The predict method will assign each row in future a predicted value which it names yhat. If you pass in historical dates, it will provide an in-sample fit. The forecast object here is a new dataframe that includes a column yhat with the forecast, as well as columns for components and uncertainty intervals.

In [ ]:
forecast = prophet_model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

- **ds**: the datestamp of the forecasted value
- **yhat**: the forecasted value of our metric (in Statistics, yhat is a notation traditionally used to represent the predicted values of a value y)
- **yhat_lower**: the lower bound of our forecasts
- **yhat_upper**: the upper bound of our forecasts

You can plot the forecast by calling the Prophet.plot method and passing in your forecast dataframe.

In [ ]:
fig1 = prophet_model.plot(forecast)

If you want to see the forecast components, you can use the Prophet.plot_components method. By default you’ll see the trend, yearly seasonality, and weekly seasonality of the time series. If you include holidays, you’ll see those here, too.

In [ ]:
# So, Let's decompose this graph into the trend and seasonality.
fig2 = prophet_model.plot_components(forecast)

It looks like the latter seasonal high and low data points in the training data weren't picked up super well. Perhpas we should try making that seasonality effect multiplicative.

# Model evaluation

In [ ]:
evaluate(peyton['y'], forecast['yhat'])

In [ ]:
# Evaluate the model's performance on test data using cross-validation
df_cv = cross_validation(prophet_model, horizon='24 day', period='30 day', 
                         initial=str(len(peyton))+' days')
df_p = performance_metrics(df_cv)
print(df_p)

## As we saw that the additive model doesn't fit the data well, So, let's try the multiplicative model.

In [ ]:
prophet_model = Prophet(seasonality_mode='multiplicative')
prophet_model.fit(peyton)

An interactive figure of the forecast and components can be created with plotly. You will need to install plotly 4.0 or above separately, as it will not by default be installed with prophet. You will also need to install the notebook and ipywidgets packages.

In [ ]:
# Prophet's make_future_dataframe method makes a DataFrame that includes the historical data, as well as the predictions.
future = prophet_model.make_future_dataframe(periods=24, freq='m')
future.tail()

In [ ]:
forecast = prophet_model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = prophet_model.plot(forecast)


In [ ]:
# So, Let's decompose this graph into the trend and seasonality.
fig2 = prophet_model.plot_components(forecast)

Looks like people travel in the summer.

In [ ]:
from fbprophet.plot import plot_plotly, plot_components_plotly

plot_plotly(prophet_model, forecast)

In [ ]:
plot_components_plotly(prophet_model, forecast)


## Model evaluation

In [ ]:
# Evaluate the model's performance on test data using cross-validation
df_cv = cross_validation(prophet_model, horizon='24 day', period='30 day', 
                         initial=str(len(peyton))+' days')
df_p = performance_metrics(df_cv)
print(df_p)

In [ ]:
peyton['y']=np.log(peyton['y'])
prophet_model = Prophet()
prophet_model.fit(peyton)
forecast = prophet_model.predict(future)
fig1 = prophet_model.plot(forecast)


In [ ]:
# So, Let's decompose this graph into the trend and seasonality.
fig2 = prophet_model.plot_components(forecast)

In [ ]:
plot_plotly(prophet_model, forecast)

Prophet respectively, the errors are significantly lower than the errors obtained with ARIMA.

## Model evaluation

In [ ]:
evaluate(peyton['y'], forecast['yhat'])

In [ ]:
# Evaluate the model's performance on test data using cross-validation
df_cv = cross_validation(prophet_model, horizon='24 day', period='30 day', 
                         initial=str(len(peyton))+' days')
df_p = performance_metrics(df_cv)
print(df_p)

the model with the multiplicative seasonality effect performs compared to a persistence model. and the model after data log transforming is more effecient.